In [1]:
%run ../widgets/config_check.py

In [2]:
# <api>
%matplotlib inline
import os
path = os.getcwd()
s = '/'
pardir = s.join(path.split(s)[:-1])

import numpy as np

# Load widgets
from jupyter_cms.loader import load_notebook
GD_widgets = load_notebook(str(pardir + '/widgets/2D_gradientdescent_widget.ipynb'))
GD_methods = load_notebook(str(pardir + '/widgets/GradientDescent_methods.ipynb'))
Widget_targets = load_notebook(str(pardir + '/widgets/Widget_targets.ipynb'))

# Introduction to gradient descent

Gradient descent is one of the main workhorses of machine learning. In its simplest form, it is a first order optimization algorithm designed to find the minimum of a differentiable function $f(\mathbf{x})$. 

The core idea is to iteratively approach a minimum (often local, but ideally global) by taking steps in direction of the negative gradient of the function at the current point. This is the direction in which $f(\mathbf{x})$ decreases fastest. If the value of the gradient isn't analytically known, then it is computed numerically.

Starting at an original guess $\mathbf{x}_0$, steps proportional to the negative gradient are taken: 

$$ \mathbf{x}_{n+1} = \mathbf{x}_n - \lambda \nabla f(\mathbf{x}_n)$$

This is based on the idea that if the step size $\lambda$ (in later machine learning algorithms this will be referred to as the learning rate) is taken to be small enough:

$$ f(\mathbf{x}_n) \geq f(\mathbf{x}_{n+1}) \approx f(\mathbf{x}_n) + \nabla f(\mathbf{x}_n)^T (\mathbf{x}_{n+1} - \mathbf{x}_n) = f(\mathbf{x}_n) - \nabla f(\mathbf{x}_n)^T \nabla f(\mathbf{x}_n), \quad \mathtt{where} \, \, \, \mathbf{x}_{n+1} = \mathbf{x}_n - \lambda \nabla f(\mathbf{x}_n) $$

and we can find a minimum of the function, such that

$$ f(\mathbf{x}_0) \geq f(\mathbf{x}_1) \geq \ldots \geq f(\mathbf{x}_n) $$

Depending on the initial guess $\mathbf{x}_0$, the minimum that is reached will either be a local or the global one. Generally, the global minimum is approached if the initial guess is already close to it, or the function is convex, in which case all local minima are global minima. If the sign is flipped and we move into the direction of the (positive) gradient, then the method will find maxima and is referred to as gradient ascent.

Note that the rate of convergence of the gradient descent method is particularly slow the closer we get to a minimum, the method often does not take the shortest direction to the minimum (think of a zig-zag like behavior) and can get stuck in poor local minima or saddle points. 

Multiple variants have therefore been proposed to address the shortcomings of gradient descent. We will delve into each variant and the advances as well as drawbacks with each notebook. 

$\textbf{Why is gradient descent so popular?}$

* Gradient descent works in very high-dimensional spaces
* If analytically unknown, derivates can often efficiently be calculated numerically. Automatic differentiation tools have been developed for this purpose. 
* A variety of extensions have been developed to overcome slow convergence (through e.g. inclusion of momentum) or to escape shallow local minima and saddle points. Other extensions address choice of locally optimal step size $\lambda$.
* Gradient descent, or specifically stochastic gradient descent, can be used to optimize a variety of machine learning models, e.g. logistic regression. If coupled with back-propagation to efficiently compute gradients, it also forms the basis for training artificial neural networks (NN).  

# Stochastic gradient descent and the machine learning perspective

While it isn't necessary for our understanding of gradient descent to introduce the machine learning perspective at this point, we will go on a small tangent so that we can introduce our code with corresponding notation. This will later make it easier to move to more advanced topics such as how to use gradient descent for the training of neural networks without making many changes. 

The above introduction assumes that we have a clear idea of the function for which we want to find a minimum, in machine learning this is needs some thought. Here, we define an objective function, or in our specific case of finding minima, a so called loss function which we seek to minimize. We typically define the loss function to be a surrogate of some desired measure to estimate the difference between an estimate and true value of a data instance. If you think of linear regression, an example could be the mean-squared error between target values and model predictions. Our task is to then learn/find a set of parameters $\mathbf{\theta}$ such that we minimize the defined loss/objective function of the form: 

$$ \mathcal{L}(\mathbf{\theta}) = \frac{1}{N} \sum_{i=1}^{N} L_i(\mathbf{\theta}) $$

where $i$ refers to the i-th data sample in an overall dataset consisting of $N$ points.
In case of the mean-squared error, the loss for data point $i$, i.e. $(\mathbf{x}_i, t_i)$, is given as $L_i(\mathbf{\theta}) = \frac{1}{2} (y(\mathbf{x}_i; \mathbf{\theta}) - t_i)^2$ where $y(\mathbf{x}_i; \mathbf{\theta})$ denotes the model predictions for input $\mathbf{x}_i$ with parameters $\mathbf{\theta}$.

In analogy to above description of gradient descent we minimize the objective by performing iterative updates: 

$$ \mathbf{\theta} = \mathbf{\theta} - \lambda \nabla \mathcal{L}(\mathbf{\theta}) = \mathbf{\theta} - \lambda \frac{1}{N} \sum_{i}^{N} \nabla L_{i}(\mathbf{\theta})$$

We refer to the case where we take an optimization/update step after evaluation of the gradient of the loss of the entire data set as gradient descent or batch gradient descent. We can make the method stochastic by sampling specific data points and taking gradient updates per individual data point. This other extreme is known as stochastic gradient descent or "on-line" gradient descent. We refer to updates on the entire batch of data (i.e. one for batch gradient descent and $N$ for stochastic gradient descent) as one epoch.

$\textbf{Some advantages and disadvantages of stochastic gradient descent:}$

+ Stochastic gradient descent has a higher update frequency and thus approaches values close to a minimum more rapidly.
+ Due to the sampling of data points and thus also gradients there exists an inherent noise component that can help overcome small local valleys or saddle points.
+ Depending on dataset size it is computationally more efficient and less memory intensive. 
- Stochastic gradient descent does not converge unless the step size/learning rate decreases with an appropriate rate over time. This in itself is a hyper-parameter that can drastically impact the optimization procedure and thus the found solution. Furthermore, it requires that the individual gradients (per data point) are unbiased estimates for the batch gradient. This is usually the case, if the loss function is additive, i.e. $\mathcal{L} = \sum_i L_i$.  

To balance advantages and disadvantages of batch and stochastic gradient descent, a mini-batch stochastic gradient descent hybrid is typically employed, where an inner sum over smaller batches over $M$ data points is introduced for the updates.

## Examples of two-dimensional functions where derivatives are known analytically

The previous section has served as a section to motivate (stochastic) gradient descent and its applications and importance for machine learning. 
For now, let us take a step back and take a look at the example of some known functions to introduce the different algorithmic variants, analyze and better understand their individual nuances. We will then move on to the application of the different gradient descent methods in machine learning algorithms. 

Here is an example of a 2-D Normal distribution where we select a starting point $\mathbf{x}_0$ and use gradient descent to find the minimum.

In [3]:
#<api>       
class Norm2D(object):
    '''2-dimensional multivariate normal distribution.'''
    
    def __init__(self, mean, cov):
        self.cov_inv = np.linalg.inv(np.array(cov))
        self.mean = mean
        self.det = cov[0][0]*cov[1][1] - cov[0][1]*cov[1][0]
        self.const = - np.log(2*np.pi) - 0.5 * np.log(self.det)
        
    def logpdf(self, xy):
        x = xy[0]
        y = xy[1]
        return (self.const - 0.5*((x-self.mean[0])**2 * self.cov_inv[0,0] + 
                                  (y-self.mean[1])**2 * self.cov_inv[1,1] +
                                  (x-self.mean[0]) * (y-self.mean[1]) * 
                                  (self.cov_inv[0,1] + self.cov_inv[1,0])))        

class MultNorm (Widget_targets.Target):
    '''Multivariate normal target distribution'''
    def __init__(self, size=3, mean=[0,0], cov=[[1, 0.99],[0.99, 1]], cmap='Viridis'):
        self.cov_inv = np.linalg.inv(np.array(cov))
        self.mean = mean
        self.cmap = cmap
        self.distr = Norm2D(mean=mean, cov=cov)
        self.init_grid(size)

    def grad(self, xy):
        '''Analytical calculation of gradient.'''
        grad = np.array([0.,0.])
        grad[0] = - self.cov_inv[0,0]*(xy[0]-self.mean[0]) - \
                    (self.cov_inv[0,1]+self.cov_inv[1,0])*(xy[1]-self.mean[1])/2
        grad[1] = - self.cov_inv[1,1]*(xy[1]-self.mean[1]) - \
                    (self.cov_inv[0,1]+self.cov_inv[1,0])*(xy[0]-self.mean[0])/2

        return grad

In [4]:
mn = MultNorm(size=3, mean=[0, 0], cov=[[1., 0.95],[0.95, 1.]])
mn.plot_surface()

Figure(axes=[Axis(num_ticks=7, scale=LinearScale(max=3.0, min=-3.0)), Axis(num_ticks=7, orientation='vertical'…

We have included the analytical gradient for the function in above code. In fact, to analyze the variants, we will work on examples where we know the analytical gradient precisely and where we will simulate stochastic gradient descent by adding a noise component to it. 

We previously mentioned, even though we are looking at values and gradients of a known function $f(\mathbf{x})$, we will keep the code API general, so that a later transfer to methods which update parameters, e.g. the learning rate $\lambda$, doesn't require fundamental changes.

If we consider batch gradient descent (often also referred to as "Vanilla" gradient descent) the basic API could look like this:

In [5]:
#<api>
class VanillaGD(GD_methods.GradientDescent):
    '''Vanilla gradient descent class'''

    def __str__(self):
        return "\nVanilla gradient descent has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)       
       
    def comp_update(self, theta):
        
        grad = self.target_GD.grad(theta)
                
        update = self.learning_rate * grad
        
        return update

We have abstracted away the detail of the class' constructor that initializes the learning rate as well as the gradient calculation $\mathbf{\theta}_{n+1} = \mathbf{\theta}_n - \Delta \mathbf{\theta}$ so that we can focus on the implementation of the update calculation $\Delta \mathbf{\theta}$. In our simplest case this corresponds to multiplication of the learning rate with the obtained gradient of our input. 

To emulate the effects of stochastic gradient descent, we add an optional noise component to the gradient calculation. This noise component follows a Normal distribution and illustrates the point that when we sample in stochastic gradient descent, the precise gradient value is not known. 

In [6]:
def grad(self, theta):
    return -1 * self.target.grad(theta) + self.stochastic * np.random.normal(loc=0., scale=self.stochastic_std) 

Let us take a look at the Vanilla gradient descent and stochastic gradient descent methods for a variety of functions in an interactive widget. 

You can drag and drop the starting point (in white) or change its position with the position sliders and change the parameters such as epochs (number of steps) and learning rate. If the tick for "stochastic gradient descent" is set, then a noise component is added to emulate stochastic gradient descent behavior.

In [7]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='VanillaGDM')
widget.show()

### Questions and suggestions for exploration

1. If we drag the starting point far away from the distributions optimum, what do we observe? What are potential parameter changes required for gradient descent to still converge to the optimum? 
2. Change the distribution to a "Bimodal Normal" distribution. Place the starting point closer to local as well as global minimum and play with the parameters. If the starting point is closer to the local minimum, is it possible for the procedure to reach the global one? 
3. Investigate the target that has a saddle point and get an empirical intuition for the gradient descent parameters for this case. If you enable stochasticity in the gradient (click the checkbox), what can you observe? 

# Variants of gradient descent

Now that we have introduced gradient descent in its basic form and the stochastic variant, there is plenty more variants that we can explore.
We provide introductions to some of those, namely: momentum, nesterov accelerated gradient descent, Adagrad, RMSProp and Adam in the [next notebook](GradientDescent_Variants.ipynb).